* `Prior` : 우리가 알기를 원하는 $Z$에 대해 알고 있는 정보. $\large p(Z)$
* `Posterior` : 우리가 가진 데이터 $X$을 가지고 구하고 싶은 확률분포 $\large p(z|x)$
* `Likelihood` : $Z$가 주어진 경우 입력값 $X$의 확률분포  $\large p(x|z)$

# Monte Carlo Method



굉장히 유명하고 자주 쓰이는 알고리즘. 하지만 그만큼 간단하다.

임의의 복잡한 분포 $p(x)$에 대해 $p(x)$의 평균을 구하고 싶은데, 앞서 우리가 배운 Variational Inference, KL Divergence와 같은 복잡한 concept을 사용해야 한다.

하지만 Expectation $\large \mathbb{E}_p[f(x)] = \int p(x)f(x)dx$ 에 대해,

무수히 (very high) 많은 sample $N$개를 sampling한다면, `Central Limit Theorem`에 따라 Expectation을 평균으로 근사할 수 있다!

$\large \mathbb{E}_p[f(x)] = \LARGE \frac{1}{N} \large \sum\limits_{i=1}^N f(x_i)$, such that $x_i \sim p(x)$

Thus, by the Central Limit Theorem,

$\large s \rightarrow N(\mu, \sigma^2) = \LARGE
\begin{cases}
    \mu = \mathbb{E}_p[f(x)] \\
    \sigma^2 = \LARGE \frac{1}{N} \large \mathbb{V}_p[f(x)]\\
\end{cases}$

으로 approximate가 가능합니다.

# Inverse Transform Sampling

$U$가 $[0, 1]$ 사이 uniform하게 배분되어 있는 continuous random variable이라면,<br>
$U$ 분포에 특정한 변환 함수 $T(U)$를 적용해 우리가 원하는 $X$ 분포로 Sampling할 수 있다.

`Inverse Transform Sampling` : $U$에서 $X$을 만든는 식 $T(U)$ 찾기

<img src = "./img/week0_6.png" style = "width : 500px;">

Let $\large T(U) = X$,<br>
$\large F_x(x) = P(X \le x) = P(T(U) \le x) = P(U \le T^{-1}(x)) = T^{-1}(x)$

$\large \therefore T(x) = F_x^{-1}(x)$

uniformly distributed distribution에 $F_x^{-1}(x)$을 대입하면 원하는 분포대로 sampling을 진행할 수 있다.

## ex) exponential distribution

$\large f(x) = \LARGE
\begin{cases}
    e^{-\lambda x}, & \text{if } x \ge 0\\
    0,              & \text{otherwise}
\end{cases}$

$\large F_x(x) = \LARGE
\begin{cases}
    1 - e^{-\lambda x}, & \text{if } x \ge 0\\
    0,              & \text{otherwise}
\end{cases}$

$\large y = 1 - e^{-\lambda x}$<br>
$\large 1 - y = e^{-\lambda x}$<br>
$\large ln(1 - y) = -\lambda x$

$\large \therefore x = \LARGE \frac{-ln(1-y)}{\lambda}$

## Limitation

분포에 알맞게 sampling하려면,

1. 해당 분포의 CDF을 구해야 하고 -> 적분불가능한 여러 함수들이 있음 (intractable)
2. 해당 CDF의 inverse도 구해야 한다 -> 또한 inverse 불가능한 함수들이 많음.

따라서 inverse transform sampling은 간단한 함수를 sampling할 때 사용하고, 다른 방법들로 더 general하게 sampling을 진행함.

# Rejection Sampling

<img src = "./img/week0_7.png" style = "width : 500px;">

위의 이미지처럼 CDF, 그리고 inverse로 나타내기 어려운 함수 $\tilde{p}(z)$에서 sampling을 진행하고자 한다면, 계산이 용이한 가우시안 분포 $q(z)$를 설정해 sampling을 진행할 수 있다.

$q(z)$는 Variance Inference에서 설정했을 때처럼 아무 가우시안 분포로 설정할 수 있지만, 본래 분포인 $\tilde{p}(z)$와 가까울 수록 좋다. $q(z)$을 정했으면 모든 domain $z$에 대해서 $kq(z) \ge \tilde{p}(z)$가 되도록 $q(z)$에 $k$를 곱해준다. $kq(z)$와 $\tilde{p}(z)$간 차이가 적을수록 성능이 올라가기 때문에 가까운 분포를 잡을수록 계산이 용이하다.

우리가 $kq(z)$을 설정하는 이유는, sampling이 어려운 $\tilde{p}(z)$을 푸는 것보다 sampling하기 쉬운 $kq(z)$의 임의의 점 $z_0$에서, sampling한 값이 $p(z_0)$ 안에 있으면 accept하고, 밖에 있으면 reject하는 방식으로 sampling을 진행한다.

Let $kq(z)$ = $g(z)$ and $\tilde{p}(z)$ = $f(z)$ ($g(z) \ge f(z)$),

1. Sample $z$ from $g(z)$
2. Accept with probability $\large \frac{f(z)}{M \times g(z)}$<br>
where $M$ = $\int f(z) dz$

### proof)

$\large p(z|A) = \LARGE \frac{p(A|z)p(z)}{p(A)} = \frac{\frac{f(z)}{M \times g(z)}\times g(z)}{p(A)}$

Since $\large p(A) = \int g(z) \LARGE \frac{f(z)}{M \times g(z)} dz = \frac{1}{M}\int f(z)dz = \frac{NC}{M}$,

$\large p(z|A) = \LARGE \frac{f(z)/M}{NC/M} = \frac{f(z)}{NC} = \large \tilde{p}(z)$

$\therefore$ Rejection Sampling을 통해 $\tilde{p}(z)$의 분포를 sampling할 수 있다!

# Importance Sampling

`Importance Sampling` 또한 앞선 `Rejection Sampling`과 유사하게 $\tilde{p}(z)$와 유사한 $q(z)$을 approximate해 계산한다. 하지만 이번 method는 `Monte Carlo` Theorem을 사용해 Expectation $\mathbb{E}$를 사용해 sampling을 진행한다는 점에서 다르다.

$\large \mathbb{E}_p[f(x)] = \int p(x)f(x)dx = \int q(x)[\LARGE \frac{p(x)}{q(z)} \large f(x)]dx = \mathbb{E}_q[\LARGE  \frac{p(x)}{q(z)} \large f(x)]$

Monte Carlo Theorem을 사용해 sampling하기 쉬운 $q$로부터 계산을 진행합니다.

$\large \mathbb{E}_q[\LARGE  \frac{p(x)}{q(z)} \large f(x)] = \LARGE \frac{1}{N} \large \sum\limits_{i=1}^N \frac{p(x_i)}{q(x_i)}f(x_i)$

이를 통해 기존 $q(x)$을 통해 근사할 때보다 더 자세한 $\frac{p(x)}{q(x)}f(x)$을 통해 sampling을 진행한다.

<img src = "./img/week0_8.png" style = "width : 500px;">